## Importing Packages

In [23]:
!pip install transformers -q

import pandas as pd
import numpy as np
import pickle as pk
from Embeddings import *

from sklearn.metrics import precision_score, recall_score, f1_score, r2_score

In [31]:
text = pd.read_csv('/content/links.csv')['text'].to_list()
query = pd.read_csv('/content/query_df.csv')['0'].to_list()

prec_recall = pd.read_csv('/content/manualData.csv')
prec_recall = prec_recall.drop(['Unnamed: 7', 'Unnamed: 8', 'Relevance1', 'Relevance2', 'Rel3', 'rel4', 'rel5'], 
                               axis = 1)
prec_recall = prec_recall[:20]
prec_recall1 = pd.read_csv('/content/manualData1.csv')

predicted1 = pd.read_csv('/content/PredictedPrec1.csv')
engine = pd.read_csv('/content/EnginePrec1.csv')

## Fuzzy Logic For Keywords - Naive Model

### Making New DataFrame

In [ ]:
columns = []
for i in range(len(prec_recall)):
    user_query = prec_recall['Query'][i]

    scores = []
    for que in text:
        scores.append(fuzz.ratio(user_query, que))

    ranks = np.array(scores).argsort()[-100:][::-1]
    reccs = [text[i] for i in ranks]
    reccs = list(set(reccs))[-10:]
    columns.append(reccs)

In [ ]:
# pd.DataFrame(columns).to_csv('/content/PredictedPrec.csv', index = False)

### Calculating Precision, Recall and F1 Score

In [6]:
def accuracy_f1_summary(y_test, y_pred):
    
    # precision tp / (tp + fp)
    precision = precision_score(y_test, y_pred)
    
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, y_pred)
    
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, y_pred)

    return precision, recall, f1

In [7]:
prec_recall1 = prec_recall1.fillna(0)
engine1 = engine1.fillna(0)

prec_recall1 = prec_recall1.drop(['Sr.No.', 'Query'], axis = 1)
predicted1 = predicted1.drop(['Sr. No.', 'Query', 'Relevance6', 'Relevance7', 'Relevance8', 'Relevance9', 'Relevance10'], axis = 1)

In [8]:
prec, rec, f11 = [], [], []
for i in range(20):
    a, b, c = accuracy_f1_summary(prec_recall1.iloc[i], predicted1.iloc[i])
    prec.append(a)
    rec.append(b)
    f11.append(c)

# Averages
print(f'Average Precision: {sum(prec)/len(prec)}')
print(f'Average Recall: {sum(rec)/len(rec)}')
print(f'Average F1 Score: {sum(f11)/len(f11)}')

Average Precision: 0.6875
Average Recall: 0.34750000000000003
Average F1 Score: 0.4305555555555555


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Recommendation Engine 

### Making new Dataframe

In [11]:
with open('/content/Embedd.pk', 'rb') as f:
    embedding = pk.load(f)

In [ ]:
columns = []
for i in range(len(prec_recall)):
    print(f'Query: {i + 1}')
    user_query = prec_recall['Query'][i]

    we_user = SentenceEmbedding(user_query)
    we_user.get_embeddings()

    scores = []
    for text in embedding:
        scores.append(r2_score(we_user.embedding, text))

    ranks = np.array(scores).argsort()[-10:][::-1]
    reccs = [query[i]for i in ranks[:5]]

    columns.append(reccs)

In [30]:
pd.DataFrame(columns).to_csv('/content/EnginePrec.csv', index = False)

### Calculating Precision and Recall


In [32]:
def accuracy_f1_summary(y_test, y_pred):
    from sklearn.metrics import precision_score, recall_score, f1_score
    
    # precision tp / (tp + fp)
    precision = precision_score(y_test, y_pred)
    
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, y_pred)
    
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, y_pred)

    return precision, recall, f1

In [34]:
engine1 = engine.fillna(0)
predicted1 = predicted1.fillna(0)

engine1 = engine1.drop(['Sr. No.', 'Query'], axis = 1)
predicted1 = predicted1.drop(['Sr. No.', 'Query', 'Relevance6', 'Relevance7', 'Relevance8', 'Relevance9', 'Relevance10'], axis = 1)

In [37]:
prec, rec, f11 = [], [], []
for i in range(20):
    a, b, c = accuracy_f1_summary(engine1.iloc[i], predicted1.iloc[i])
    prec.append(a)
    rec.append(b)
    f11.append(c)

# Averages
print(f'Average Precision: {sum(prec)/len(prec)}')
print(f'Average Recall: {sum(rec)/len(rec)}')
print(f'Average F1 Score: {sum(f11)/len(f11)}')

Average Precision: 0.6583333333333334
Average Recall: 0.4208333333333333
Average F1 Score: 0.4824206349206349


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
